In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (15, 10)

import seaborn as sns
from scipy import stats
!pip install statsmodels
import statsmodels.api as sm
import time

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
app_train = pd.read_csv("app_train_red.csv")
app_test = pd.read_csv("app_test_red.csv")
prev_app = pd.read_csv("previous_application.csv")

In [3]:
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
prev_app.NAME_CONTRACT_STATUS.value_counts()

Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64

In [5]:
# Were current customers approved for a previous loan?
print(app_train.SK_ID_CURR.isin(prev_app.SK_ID_CURR).value_counts())
print(app_test.SK_ID_CURR.isin(prev_app.SK_ID_CURR).value_counts())

True     291057
False     16454
Name: SK_ID_CURR, dtype: int64
True     47800
False      944
Name: SK_ID_CURR, dtype: int64


Not everyone has previously applied for a credit line from Home Credit thus there are quite a few new borrowers, 16454 in fact. Makes sense given the areas Home Credit usually services, meaning since they primarily service Asian countries which tend to be largely cash based blah blah something about them not really using banks and there being the potential for a lot of new customers for lenders (emerging new markets).

In [6]:
# How many have the current customers applied for loans before? What is their approval rate?
approval_rate = prev_app.groupby("SK_ID_CURR").NAME_CONTRACT_STATUS.agg(total_apps = "count").reset_index()
approval_rate.total_apps

0         1
1         1
2         3
3         1
4         2
         ..
338852    1
338853    1
338854    2
338855    2
338856    8
Name: total_apps, Length: 338857, dtype: int64

In [27]:
approval_rate = pd.pivot_table(prev_app, index = ["SK_ID_CURR"], columns = ["NAME_CONTRACT_STATUS"], values = "AMT_ANNUITY", aggfunc = len, fill_value=0)

In [20]:
rates = approval_rate.apply(lambda x: x/sum(x), axis = 1)
rates.columns = rates.columns + "_rate"

In [28]:
approval_rate["total_apps"] = approval_rate.sum(axis = 1)
approval_rate
approval_rate = approval_rate.join(rates)

In [29]:
approval_rate.head()

NAME_CONTRACT_STATUS,Approved,Canceled,Refused,Unused offer,total_apps,Approved_rate,Canceled_rate,Refused_rate,Unused offer_rate
SK_ID_CURR,,,,,,,,,
100001,1,0,0,0,1,1.0,0.0,0.0,0.0
100002,1,0,0,0,1,1.0,0.0,0.0,0.0
100003,3,0,0,0,3,1.0,0.0,0.0,0.0
100004,1,0,0,0,1,1.0,0.0,0.0,0.0
100005,1,1,0,0,2,0.5,0.5,0.0,0.0


In [52]:
# prev_applicant_train = 
pd.DataFrame(app_train.SK_ID_CURR.isin(prev_app.SK_ID_CURR)).replace({"True":1}, {"False":0})

,SK_ID_CURR
0,True
1,True
2,True
3,True
4,True
...,...
307506,True
307507,True
307508,True
307509,True


In [48]:
help(pd.DataFrame.replace)

Help on function replace in module pandas.core.frame:

replace(self, to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad')
    Replace values given in `to_replace` with `value`.
    
    Values of the DataFrame are replaced with other values dynamically.
    This differs from updating with ``.loc`` or ``.iloc``, which require
    you to specify a location to update with some value.
    
    Parameters
    ----------
    to_replace : str, regex, list, dict, Series, int, float, or None
        How to find the values that will be replaced.
    
        * numeric, str or regex:
    
            - numeric: numeric values equal to `to_replace` will be
              replaced with `value`
            - str: string exactly matching `to_replace` will be replaced
              with `value`
            - regex: regexs matching `to_replace` will be replaced with
              `value`
    
        * list of str, regex, or numeric:
    
            - First, if `to_replace

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9263f292-ea73-48e7-8892-f3d3ed1db03c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>